In [ ]:
import hoss
import os
import shutil
import time

## Connect to local server
This notebook demonstrates basic operations using a single Hoss server.

**Note, for this notebook to work, it's assumed you're running against the system running in
dev mode and able to connect to localhost. If you are not, you need an admin to create additional
buckets or already have namespaces that you can use and sync between.**

We start by connecting the the "local" server. If using a different server be sure to change the `.connect()` arg

In [ ]:
server_local = hoss.connect('http://localhost')

## Create namespaces for the sync test
Syncing is done between namespaces, so we create two test namespaces that can be configured

In [ ]:
# When running minio locally, we can simply create a folder in the minio data directory
# to create a new bucket
hoss_data_dir = os.path.expanduser(os.path.join("~", ".hoss", "data", "nas"))
os.makedirs(os.path.join(hoss_data_dir, "src_bucket"))
os.makedirs(os.path.join(hoss_data_dir, "tgt_bucket"))

In [ ]:
ns_src = server_local.create_namespace("ns-src", "test source namespace", "default", "src_bucket")
ns_tgt = server_local.create_namespace("ns-tgt", "test target namespace", "default", "tgt_bucket")

## Enable namespace syncing

Set the namespaces to duplex syncing. That will let us then configure individual datasets
for 1-way (simplex) or 2-way (duplex) syncing

In [ ]:
# After enabling duplex syncing, we need to wait 45 seconds. This is due to the reload cycle
# on the sync service that has to happen twice, once for each direction of the sync relationship
ns_src.enable_sync_target("http://localhost", "duplex", "ns-tgt")
print("Waiting 45 seconds for sync service reloads")
time.sleep(45)

## Create a source dataset

Create a source dataset to write objects to

In [ ]:
ds_src = ns_src.create_dataset("sync-test", "A dataset for a sync testing")
ds_src.display()

In [ ]:
# Dataset should not exist in the target namespace yet
try:
    ns_tgt.get_dataset("sync-test")
    print("Dataset does exist in the target")
except hoss.error.NotFoundException:
    print("Dataset does not exist in the target yet")

## Enable 1-way syncing on the dataset
Enable 1-way syncing on the dataset. Now every file you write to the source dataset
will automatically be synced to the same dataset in the target namespace.

In [ ]:
ds_src.enable_sync("simplex")
assert ds_src.is_sync_enabled() is True

print("Waiting 25 seconds for sync service reload")
time.sleep(25)

In [ ]:
# Dataset should exist in the target now
ds_tgt = ns_tgt.get_dataset("sync-test")

## Write files in the source and they should appear in the target

In [ ]:
f_src = ds_src / "my-file.txt"
f_tgt = ds_tgt / "my-file.txt"


assert f_tgt.exists() is False
f_src.write_text("Hello, Hoss!")

In [ ]:
time.sleep(5)
assert f_tgt.exists() is True

In [ ]:
print(f_tgt.read_text())

## Write files in the target and they should NOT sync to the source

In [ ]:
f2_src = ds_src / "my-file2.txt"
f2_tgt = ds_tgt / "my-file2.txt"


assert f2_src.exists() is False
f2_tgt.write_text("Hello, Hoss!")

In [ ]:
time.sleep(5)
assert f2_tgt.exists() is True
assert f2_src.exists() is False

## Enable 2-way syncing on the dataset

Enable 2-way syncing on the dataset. Now every file you write to the source dataset
will automatically be synced to the same dataset in the target namespace. Also,
every file you write in the dataset in the target namespace will be synced
back to the dataset in the source namespace

In [ ]:
ds_src.disable_sync()
print("Waiting 25 seconds for sync service reload")
time.sleep(25)
assert ds_src.is_sync_enabled() is False

ds_src.enable_sync("duplex")
assert ds_src.is_sync_enabled() is True

print("Waiting 25 seconds for sync service reload")
time.sleep(30)

## Write files in the source and they should appear in the target

In [ ]:
f3_src = ds_src / "my-file3.txt"
f3_tgt = ds_tgt / "my-file3.txt"


assert f3_tgt.exists() is False
f3_src.write_text("Hello, 2-way syncing Hoss!")

In [ ]:
time.sleep(5)
assert f3_tgt.exists() is True

In [ ]:
print(f3_tgt.read_text())

## Write files in the target and they should also sync to the source

In [ ]:
f4_src = ds_src / "my-file4.txt"
f4_tgt = ds_tgt / "my-file4.txt"


assert f4_src.exists() is False
f4_tgt.write_text("Hello, Hoss!")

In [ ]:
time.sleep(5)
assert f4_tgt.exists() is True
assert f4_src.exists() is True

## Clean up this example
Run these cells to remove the resources created during the test

In [ ]:
ds_src.disable_sync()
time.sleep(25)

In [ ]:
ns_src.disable_sync_target("http://localhost", "ns-tgt")
time.sleep(25)

In [ ]:
ns_src.delete_dataset("sync-test")

In [ ]:
ns_tgt.delete_dataset("sync-test")

In [ ]:
server_local.delete_namespace("ns-src")

In [ ]:
server_local.delete_namespace("ns-tgt")

In [ ]:
shutil.rmtree(os.path.join(hoss_data_dir, "src_bucket"), ignore_errors=True)
shutil.rmtree(os.path.join(hoss_data_dir, "tgt_bucket"), ignore_errors=True)